In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [44]:
spark = SparkSession.builder.appName('TransformationRegionSpark').getOrCreate()

In [45]:
df_region = spark.read.csv('storage/PRECOS SEMESTRAIS - AUTOMOTIVOS.csv', header=True, inferSchema=True, sep=';')

distinct_region = df_region.select('Regiao - Sigla').distinct().collect()

for row in distinct_region:
    region = row['Regiao - Sigla']
    df_region_filter = df_region.filter(col('Regiao - Sigla') == region)
    df_region_filter = df_region_filter.withColumn('Data da Carga ETL', current_date())
    df_region_filter = df_region_filter.withColumn('Valor de Venda', regexp_replace(col('Valor de Venda'), ',', '.'))
    
    #Exibe registros que foram segregados por região 
    df_region_filter.show()
    #Salva registros segregandos por região
    df_region_filter.write.csv(f'storage/region/{region}/full/', header=True, mode="overwrite")

    #Agregando registros por região e demais
    df_region_agg = df_region_filter.groupBy(['Regiao - Sigla', 'Data da Coleta', 'Produto', 'Bandeira']).agg(round(avg('Valor de Venda'), 2).alias('Valor de Venda'))
    #Exibe registros que foram agregados
    df_region_agg.show()
    #Salva registros agregados por região
    df_region_agg.write.csv(f'storage/region/{region}/agg/', header=True, mode="overwrite")

spark.stop()

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-------------------+------------------+---------+------------------+--------------+--------------+---------------+-----------------+-------------+-----------------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|        Complemento|            Bairro|      Cep|           Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|     Bandeira|Data da Carga ETL|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+-------------------+------------------+---------+------------------+--------------+--------------+---------------+-----------------+-------------+-----------------+
|            NE|            AL|ARAPIRACA|COMERCIAL DE COMB...| 02.817.655/0001-82|RODOVIA AL 220 - ...|       S/N|               null|          PLANALTO|57308-000|